<a href="https://colab.research.google.com/github/ahmednasri05/ImageCaptionSharedTask2025/blob/main/FineTuneModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Verify that you have the GPU recognized, this code needs GPU.
!nvidia-smi

Fri Jul 18 15:18:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
#Clone from the Github SinaLab/ImageCaptionSharedTask2025.git
!git clone https://github.com/ahmednasri05/ImageCaptionSharedTask2025.git

Cloning into 'ImageCaptionSharedTask2025'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 111 (delta 51), reused 65 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (111/111), 66.38 KiB | 3.90 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [9]:
#Change directory to the base directory
import os
os.chdir('/content/ImageCaptionSharedTask2025/ImageValFinetune')



In [ ]:
#  Install dependencies
!pip install -r requirements.txt

In [ ]:
# You have two options for uploading the training dataset:
# Option 1: Place the image folder at the following path in your Google Drive:
#           /content/drive/MyDrive/ImageVal/Train/images
#           And place the caption file here:
#           /content/drive/MyDrive/ImageVal/Train/TrainSubtask2.xlsx
# Option 2: Use the dataset via the Hugging Face link.

'''
          Please hange the directory and the token
          Please make sure to run on a pproved account
          import os
          from huggingface_hub import snapshot_download
          login(token="add you hugging face directory")

          target_dir = "/content/Train/"

          os.makedirs(target_dir, exist_ok=True)

          dataset_dir = snapshot_download(
              repo_id="SinaLab/ImageEval2025Task2TrainDataset",
              repo_type="dataset",
              local_dir=target_dir,
              local_dir_use_symlinks=False
          )

'''
from google.colab import drive
import os
from finetune_trainer import ArabicImageCaptionTrainer

# Mount Google Drive
drive.mount('/content/drive')

# Set base directory
base_dir = "/content/drive/MyDrive/ImageVal"

# Initialize trainer
trainer = ArabicImageCaptionTrainer(base_dir=base_dir)

# Setup environment
if not trainer.setup_environment():
    print(" Setup failed")
else:
    # Set expected paths
    excel_file = f"{base_dir}/Train/TrainSubtask2.xlsx"
    images_dir = f"{base_dir}/Train/images"

    if os.path.exists(excel_file) and os.path.exists(images_dir):
        if trainer.prepare_dataset(excel_file=excel_file, images_dir=images_dir):
            print(" Dataset preparation complete")
            standard_config = trainer.create_training_config(conservative=False)
            conservative_config = trainer.create_training_config(conservative=True)

            print(f" Ready to train!")
            print(f"trainer.start_training('{standard_config}')")
            print(f"or")
            print(f"trainer.start_training('{conservative_config}')  # for limited VRAM")
        else:
            print("Dataset preparation failed")
    else:
        print("Required files not found. Please upload them first.")




In [ ]:
# Model Training
!python run_training.py --base_dir /content/drive/MyDrive/ImageVal --conservative

In [ ]:
# Model Evaluation
#For evaluation last checkpoint
!python evaluate_model.py --base_dir /content/drive/MyDrive/ImageVal
